In [1]:
import dualcodec
import torch

In [5]:
model_id = "12hz_v1" # select from available Model_IDs, "12hz_v1" or "25hz_v1"
dualcodec_model_path = "./output_checkpoints/dualcodec_12hz_16384_4096_8vq_scratch/checkpoint/epoch-0000_step-0002400_loss-160.312714-dualcodec_12hz_16384_4096_8vq_scratch"
dualcodec_model = dualcodec.get_model(model_id, dualcodec_model_path)
dualcodec_inference = dualcodec.Inference(dualcodec_model=dualcodec_model, device="cuda")

Loading model from output_checkpoints/dualcodec_12hz_16384_4096_8vq_scratch/checkpoint/epoch-0000_step-0002400_loss-160.312714-dualcodec_12hz_16384_4096_8vq_scratch/dualcodec_12hz_16384_4096.safetensors
Model loaded


Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

In [6]:
# do inference for your wav
import torchaudio
audio, sr = torchaudio.load("sam.wav")
# resample to 24kHz
audio = torchaudio.functional.resample(audio, sr, 24000)
audio = audio.reshape(1,1,-1)
audio = audio.to("cuda")
# extract codes, for example, using 8 quantizers here:
semantic_codes, acoustic_codes = dualcodec_inference.encode(audio, n_quantizers=8)


In [7]:
# semantic_codes shape: torch.Size([B, 1, T])
# acoustic_codes shape: torch.Size([B, n_quantizers-1, T])



# produce output audio
out_audio = dualcodec_inference.decode(semantic_codes, acoustic_codes)

# save output audio
torchaudio.save("out.wav", out_audio.cpu().squeeze(0), 24000)
from IPython.display import Audio
Audio("out.wav")

In [ ]:
group_size = 8

#step 1 crop the tensors so they fit in group size

semantic_codes_cropped = semantic_codes[:, :, :semantic_codes.shape[2] - semantic_codes.shape[2] % group_size]
acoustic_codes_cropped = acoustic_codes[:, :, :acoustic_codes.shape[2] - acoustic_codes.shape[2] % group_size]

all_audios_list = []
for i in range(acoustic_codes.shape[2]-group_size):  
# for i in range(acoustic_codes.shape[2] - group_size-1):  
  semantic_codes_segment = semantic_codes_cropped[:, :, i:i + group_size]
  acoustic_codes_segment = acoustic_codes_cropped[:, :, i:i + group_size]

  out_audio = dualcodec_inference.decode(semantic_codes_segment, acoustic_codes_segment)

  midpoint = out_audio.shape[2] // 2
  delta = out_audio.shape[2]//(group_size*2)

  out_audio = out_audio[:, :, midpoint-delta:midpoint+delta]

  all_audios_list.append(out_audio.cpu().squeeze(0))


all_audios = torch.cat(all_audios_list, dim=1)

# convert to numpy and display

Audio(all_audios.numpy(), rate=24000)